In [24]:
import gpytorch
import pyreadr
import pooling_params as gtp

In [15]:
import numpy as np

In [2]:
import os

In [21]:
def fake_data(n):
    #temperature, last steps, prior,location
    return [[np.random.random()*50+10,np.random.random()*30,np.random.random()*20,int(np.random.random()>.5)] for i in range(n)]

In [22]:
data = fake_data(100)

In [30]:
def initialize_policy_params_TS(standardize=False,baseline_features=None,psi_features=None,\
                                responsivity_keys=None):
    #,'location_1','location_2','location_3'
    #'continuous_temp',
    global_p =gtp.TS_global_params(21,baseline_features=baseline_features,psi_features=psi_features, responsivity_keys= responsivity_keys)
    #personal_p = pp.TS_personal_params()
    #global_p =gtp.TS_global_params(10,context_dimension)
    
    
    
    #global_p.mu_dimension = 64
    
    global_p.kdim =24
    #194
    global_p.baseline_indices = [i for i in range(3+ 3*len(baseline_features))]
    #[i for i in range(192)]
    #[0,1,2,3,4,5,6]
    global_p.psi_indices = [0] + [1+baseline_features.index(j) for j in psi_features] \
    + [len(baseline_features)+1] + [(2+len(baseline_features))+baseline_features.index(j) for j in psi_features]
    #[0,64]
    global_p.user_id_index =0
    
    global_p.psi_features =psi_features
    #[0,64]
    
    #print(global_p.psi_indices )
    
    #global_p.update_period = update_period
    
    global_p.standardize = standardize

    initial_context = [0 for i in range(global_p.theta_dim)]
    
    global_p.mus0= global_p.get_mu0(initial_context)
    #global_p.get_mu0(initial_context)
    global_p.mus1= global_p.get_mu1(global_p.num_baseline_features)
    global_p.mus2= global_p.get_mu2(global_p.num_responsivity_features)
    #np.array([.120,3.3,-.11])
    #global_p.get_mu2(global_p.num_responsivity_features)
    
    #global_p.sigmas0= global_p.get_asigma(len( personal_p.mus0[person]))
    global_p.sigmas1= global_p.get_asigma(global_p.num_baseline_features+1)
    global_p.sigmas2= global_p.get_asigma( global_p.num_responsivity_features+1)
    

    global_p.mu2_knot = np.array([0]+[0 for i in range(global_p.num_responsivity_features)])
    global_p.mu1_knot = np.zeros(global_p.num_baseline_features+1)
    global_p.sigma1_knot = np.eye(global_p.num_baseline_features+1)
    global_p.sigma2_knot = np.eye(global_p.num_responsivity_features+1)

    return global_p

In [31]:
glob = initialize_policy_params_TS(standardize=False,baseline_features=[i for i in range(4)],psi_features=[],responsivity_keys=[i for i in range(4)])
               

In [34]:
glob.psi_indices

[0, 5]

In [10]:
result = pyreadr.read_r('../../data/usertest/history.RData')

In [13]:
result['data.history']

,day,decision.time,availability,probability,action,reward,dosage,engagement,work.location,other.location,variation,temperature,logpresteps,sqrt.totalsteps,prepoststeps,deliever,appclick,random.number
0,-7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.853638,NaN,381.0,NaN,NaN,NaN
1,-7.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.832306,NaN,309.0,NaN,NaN,NaN
2,-7.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.119231,NaN,1922.0,NaN,NaN,NaN
3,-7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.824306,NaN,373.0,NaN,NaN,NaN
4,-7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.693147,NaN,0.0,NaN,NaN,NaN
5,-6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.561902,76.915538,1933.0,NaN,NaN,NaN
6,-6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.404927,76.915538,1118.0,NaN,NaN,NaN
7,-6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.277145,76.915538,26.0,NaN,NaN,NaN
8,-6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.657739,76.915538,306.0,NaN,NaN,NaN
9,-6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.363099,76.915538,88.0,NaN,NaN,NaN
